## Imports

In [1]:
import os
import shutil
import tempfile
import json

import matplotlib.pyplot as plt

from monai.inferers import sliding_window_inference
from monai.config import print_config
from sklearn.model_selection import KFold

import torch
#import pytorch_lightning
import lightning
from lightning.pytorch.callbacks.model_checkpoint import ModelCheckpoint
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

from monai_unet_cv import Net


In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device = ", device)
torch.backends.cudnn.benchmark = True

device =  cuda


In [3]:
PATH = "E:\\Visual Studio\\Task01_BrainTumour\\"
K_FOLDS = 5

## Run the training

In [4]:
torch.set_float32_matmul_precision('medium')
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [5]:
with open(PATH + "dataset.json", 'r') as f:
    data = json.load(f)

test_size = int(0.1 * len(data["training"]))
print(data["training"])
test_data = data["training"][:test_size]
print(len(test_data))
train_val_data = data["training"][test_size:]
print(len(train_val_data))
kf = KFold(n_splits=K_FOLDS)
for i, (train_index, val_index) in enumerate(kf.split(train_val_data), 1):
    data['training'] = [train_val_data[idx] for idx in train_index]
    data['validation'] = [train_val_data[idx] for idx in val_index]
    data['test'] = test_data
    print(data)
    # Save the new split into a JSON file
    output_file = f'E:\\Visual Studio\\filedump\\dataset_kfold_split_{i}.json'
    with open(output_file, 'w') as outfile:
        json.dump(data, outfile, indent=4)

    print(f"Saved: {output_file}")




[{'image': './imagesTr/BRATS_457.nii.gz', 'label': './labelsTr/BRATS_457.nii.gz'}, {'image': './imagesTr/BRATS_306.nii.gz', 'label': './labelsTr/BRATS_306.nii.gz'}, {'image': './imagesTr/BRATS_206.nii.gz', 'label': './labelsTr/BRATS_206.nii.gz'}, {'image': './imagesTr/BRATS_449.nii.gz', 'label': './labelsTr/BRATS_449.nii.gz'}, {'image': './imagesTr/BRATS_318.nii.gz', 'label': './labelsTr/BRATS_318.nii.gz'}, {'image': './imagesTr/BRATS_218.nii.gz', 'label': './labelsTr/BRATS_218.nii.gz'}, {'image': './imagesTr/BRATS_434.nii.gz', 'label': './labelsTr/BRATS_434.nii.gz'}, {'image': './imagesTr/BRATS_365.nii.gz', 'label': './labelsTr/BRATS_365.nii.gz'}, {'image': './imagesTr/BRATS_265.nii.gz', 'label': './labelsTr/BRATS_265.nii.gz'}, {'image': './imagesTr/BRATS_214.nii.gz', 'label': './labelsTr/BRATS_214.nii.gz'}, {'image': './imagesTr/BRATS_314.nii.gz', 'label': './labelsTr/BRATS_314.nii.gz'}, {'image': './imagesTr/BRATS_445.nii.gz', 'label': './labelsTr/BRATS_445.nii.gz'}, {'image': './im

In [6]:
with open(PATH + "dataset.json", 'r') as f:
    data = json.load(f)

test_size = int(0.1 * len(data["training"]))
print(data["training"])
test_data = data["training"][:test_size]
print(len(test_data))
train_val_data = data["training"][test_size:]
print(len(train_val_data))
kf = KFold(n_splits=K_FOLDS)
for i, (train_index, val_index) in enumerate(kf.split(train_val_data), 1):
    data['training'] = [train_val_data[idx] for idx in train_index]
    data['validation'] = [train_val_data[idx] for idx in val_index]
    data['test'] = test_data

    # initialise the LightningModule
    net = Net(device=device, data=data, in_channels=4, out_channels=4, augmentation=False)

    # set up checkpoints
    # TODO model callback set when val_loss lowest
    checkpoint_callback = ModelCheckpoint(dirpath="./heart_data_lightning_models/", filename='{epoch}-{val_loss:.2f}-{val_dice:.2f}', monitor="val_dice", mode="max")
    early_stopping_callback = EarlyStopping(monitor="val_loss", mode="min", patience=5)
    max_epochs = 100

    # initialise Lightning's trainer.
    trainer = lightning.Trainer(
        devices=[0],
        max_epochs=max_epochs,
        callbacks=[checkpoint_callback, early_stopping_callback],
        log_every_n_steps=4,
    )

    # train
    trainer.fit(net)

[{'image': './imagesTr/BRATS_457.nii.gz', 'label': './labelsTr/BRATS_457.nii.gz'}, {'image': './imagesTr/BRATS_306.nii.gz', 'label': './labelsTr/BRATS_306.nii.gz'}, {'image': './imagesTr/BRATS_206.nii.gz', 'label': './labelsTr/BRATS_206.nii.gz'}, {'image': './imagesTr/BRATS_449.nii.gz', 'label': './labelsTr/BRATS_449.nii.gz'}, {'image': './imagesTr/BRATS_318.nii.gz', 'label': './labelsTr/BRATS_318.nii.gz'}, {'image': './imagesTr/BRATS_218.nii.gz', 'label': './labelsTr/BRATS_218.nii.gz'}, {'image': './imagesTr/BRATS_434.nii.gz', 'label': './labelsTr/BRATS_434.nii.gz'}, {'image': './imagesTr/BRATS_365.nii.gz', 'label': './labelsTr/BRATS_365.nii.gz'}, {'image': './imagesTr/BRATS_265.nii.gz', 'label': './labelsTr/BRATS_265.nii.gz'}, {'image': './imagesTr/BRATS_214.nii.gz', 'label': './labelsTr/BRATS_214.nii.gz'}, {'image': './imagesTr/BRATS_314.nii.gz', 'label': './labelsTr/BRATS_314.nii.gz'}, {'image': './imagesTr/BRATS_445.nii.gz', 'label': './labelsTr/BRATS_445.nii.gz'}, {'image': './im

TypeError: Net.__init__() got an unexpected keyword argument 'data'

## Plot the loss and dice metric

In [ ]:
plt.figure("train", (12, 6))
plt.subplot(1, 3, 1)
plt.title("Average Loss per Epoch Training")
x = [i for i in range(0,len(net.epoch_train_loss))]
y = net.epoch_train_loss
plt.xlabel("Epochs")
plt.plot(x, y)
plt.subplot(1, 3, 2)
plt.title("Average Loss per Epoch Validation")
# skip first epoch for validation loss as it is very high
x = [i for i in range(0,len(net.epoch_val_loss)-1)]
y = net.epoch_val_loss[1:]
plt.xlabel("Epochs")
plt.plot(x, y)
plt.subplot(1, 3, 3)
plt.title("Val Mean Dice")
x = [i for i in range(0,len(net.metric_values))]
y = net.metric_values
plt.xlabel("Epochs")
plt.plot(x, y)
plt.show()

## Test model on unseen data

In [ ]:
# this will print the avg loss and dice metric on the test set
trainer.test()

## Load model from checkpoint to plot predictions

In [ ]:
model = Net.load_from_checkpoint(checkpoint_callback.best_model_path, device = device, in_channels=4, out_channels=4, augmentation=False, datadir = PATH)
model.eval()
model.to(device)
model.prepare_data()

In [ ]:
# make predictions
images = []
outputs = []
labels = []
val_loader = model.test_dataloader()
#TODO only make predictions for some images?
i = 0
for val_data in val_loader: 
    if i ==2:
        break
    val_inputs, val_labels = (
        val_data["image"].to(device),
        val_data["label"].to(device),
    )
    images.append(val_inputs)
    outputs.append(model.forward(val_inputs))
    labels.append(val_labels)
    i += 1

In [ ]:
def plot_predictions(idx_of_image, outputs, images, labels):
    # Assuming outputs, labels, and images are your variables
    output = outputs[idx_of_image]
    label = labels[idx_of_image][0][0]
    image = images[idx_of_image][0][0]

    # Set the slice range from 45 to 50
    slice_start = 45
    slice_end = 55

    # Calculate the number of slices
    num_slices = slice_end - slice_start

    # Create a subplot with 'num_slices' rows and 3 columns
    plt.figure(figsize=(15, 5 * num_slices))

    for i in range(num_slices):
        # Increment the slice index
        current_slice = slice_start + i
        
        # Plot the Model Output
        plt.subplot(num_slices, 3, 3 * i + 1)
        with torch.no_grad():
            plt.imshow(torch.argmax(output.cpu(), dim=1)[0, :, :, current_slice])
        plt.title(f'Model Output - Slice {current_slice}')

        # Plot the Label
        plt.subplot(num_slices, 3, 3 * i + 2)
        plt.imshow(label.cpu()[:, :, current_slice])
        plt.title(f'Label - Slice {current_slice}')

        # Plot the Input Image
        plt.subplot(num_slices, 3, 3 * i + 3)
        plt.imshow(image.cpu()[:, :, current_slice])
        plt.title(f'Input Image - Slice {current_slice}')

    # Adjust layout for better visualization
    plt.tight_layout()

    # Show the plots
    plt.show()

## Plot predictions

In [ ]:
plot_predictions(0, outputs, images, labels)